In [3]:
from imutils.perspective import four_point_transform
from imutils import contours
import numpy as np
import argparse
import imutils
import cv2

In [33]:
image = cv2.imread("omr_test_12.jpg")

In [96]:
#image

In [35]:
len(image)

3508

In [36]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [95]:
#gray

In [38]:
len(gray[0])

2550

In [39]:
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

In [40]:
edged = cv2.Canny(blurred, 75, 200)

In [94]:
#edged

In [61]:
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [63]:
cnts = imutils.grab_contours(cnts)

In [93]:
#cnts

In [65]:
len(cnts)

64

In [66]:
if len(cnts) > 0:
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)    
        if len(approx) == 4:
            break

In [92]:
#approx

In [68]:
paper = four_point_transform(image, approx.reshape(4,2))

In [72]:
warped = four_point_transform(gray, approx.reshape(4,2))

In [91]:
#warped

In [74]:
paper = cv2.resize(paper,(500, 642))

In [76]:
thresh = cv2.threshold(warped, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

In [78]:
resized_image = cv2.resize(thresh,(500,642)).copy()

In [80]:
cropped_image = resized_image[341:641, 3:166].copy() # for resized to 500 image
cropped_image_2 = resized_image[341:641, 168:335].copy()
cropped_image_3 = resized_image[341:641, 338:497].copy()

In [97]:
cnts = cv2.findContours(cropped_image.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#cnts = cnts[0] if imutils.is_cv2() else cnts[1]

In [98]:
cnts = imutils.grab_contours(cnts)

In [ ]:
#cnts

In [104]:
questionCnts = []
for c in cnts:
    (cx, cy, cw, ch) = cv2.boundingRect(c)
    car = cw / float(ch)
    if cw >= 9 and ch >= 9 and car >= 0.9 and car <= 1.1:
        questionCnts.append(c)

In [ ]:
#questionCnts

In [106]:
len(questionCnts)

47

In [107]:
questionCnts = contours.sort_contours(questionCnts, method="top-to-bottom")[0]

In [110]:
ANSWER_KEY = {0:1, 1:4, 2:3, 3:3, 4:1, 5:1, 6:4, 7:0, 8:3, 9:1}

In [148]:
correct = 0
for (q, i) in enumerate(np.arange(0, len(questionCnts), 5)):
   
    cnts = contours.sort_contours(questionCnts[i:i+5])[0]
    
    bubbled = None
    for (j, c) in enumerate(cnts):
        
        mask = np.zeros(cropped_image.shape, dtype="uint8")
        cv2.drawContours(mask, [c], -1, 255, -1)
        mask = cv2.bitwise_and(cropped_image, cropped_image, mask=mask)
        total = cv2.countNonZero(mask)
        if bubbled is None or total > bubbled[0]:
            bubbled = (total, j)
        color = (0, 0, 255)
        k = ANSWER_KEY[q]
        #print(k,bubbled[1])
        if k == bubbled[1]:
            print('Inside',k,bubbled[1])
            color = (0, 255, 0)
            correct += 1
            
        [cx, cy, cw, ch] = cv2.boundingRect(cnts[k])
        cv2.drawContours(paper, [cnts[k]], -1, color, 2, offset=(3, 341))

Inside 3 3
Inside 3 3
Inside 0 0
Inside 0 0
Inside 0 0
Inside 0 0
Inside 0 0
Inside 1 1


In [112]:
mask = np.zeros(cropped_image.shape, dtype="uint8")

In [113]:
mask

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [114]:
len(mask)

300

In [115]:
enumerate(cnts)